# Imports

In [1]:
# data
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# machine learning
import sklearn
import lightgbm

# other
import gc
import json
import random
from numerapi import NumerAPI

# Download data

In [2]:
napi = NumerAPI()
current_round = napi.get_current_round()

filenames = napi.list_datasets()

# napi.download_dataset('v4/features.json')
# napi.download_dataset('v4/train.parquet')
# napi.download_dataset('v4/validation.parquet')
# napi.download_dataset('v4/live.parquet', f'v4/live_{current_round}.parquet')

In [3]:
feature_metadata = json.load(open('v4/features.json', 'r'))

for k, v in feature_metadata["feature_sets"].items():
    print(k, len(v))

# features = list(feature_metadata['feature_stats'].keys())
features = feature_metadata["feature_sets"]["medium"]
read_columns = features + ['era', 'data_type', 'target_nomi_v4_20']

live_data = pd.read_parquet(f'v4/live_{current_round}.parquet', columns = read_columns)
validation_data = pd.read_parquet('v4/validation.parquet', columns = read_columns)
# training_data = pd.read_parquet('v4/train.parquet', columns=read_columns)

small 38
medium 472
v2_equivalent_features 304
v3_equivalent_features 1050
fncv3_features 420
